# Indicadores de Trazabilidad 



## Importación de datos, librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Importa datos de población regional y crea DF


df_poblacion = pd.read_csv('data/aysen_population.csv') #Archivo ubicado dentro de la carpeta data

df_poblacion.head(2)

,Unnamed: 0,orden,nombre_region,codigo_region,nombre_provincia,codigo_provincia,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,0,7172,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Coyhaique,11101,28763,29055,57818
1,1,7194,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Lago Verde,11102,494,358,852


In [3]:
# Importa datos de comunas corregidas por establecimiento de salud

df_establecimiento = pd.read_excel('data/establecimientos_salud_dict.xlsx') #Archivo ubicado dentro de la carpeta data


df_establecimiento.head()

,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
0,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique
1,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique
2,Centro Comunitario de Salud Familiar Villa Ma...,Aysen,Aysen
3,Centro Comunitario de Salud Familiar Alejandro...,Coyhaique,Coyhaique
4,Centro Comunitario de Salud Familiar Puerto Ch...,Aysen,Aysen


In [4]:
# Importa datos desde la base de datos recivida via FTP
## PRIMERO: Guardar archivo dentro de la carpeta data
## SEGUNDO: No cambiar nombre ni formato del archivo recibido
## TERCERO: RECORDAR CAMBIAR FECHA EN EL ARCHIVO A IMPORTAR A CONTINUACION, i.e: 20210829 para este domingo 29 de Agosto 2021

df =  pd.read_csv('data/20210830_Región de Aysén del General Carlos Ibáñez del Campo_notificaciones_0600.csv', 
                   sep='~', # CSV con separador ''~''
                   low_memory=False
                  )

##  limpieza y ordenamiento 

In [5]:
# Une datos con diccionario comunal

dff = df.merge(df_establecimiento, how='left')

In [6]:
# Integra datos poblacionales

df_merged = dff.merge(df_poblacion.drop(columns= ['Unnamed: 0','orden','nombre_region', 'codigo_region', 'nombre_provincia','codigo_provincia']), left_on='comuna_seremi_xi', right_on='nombre_comuna', how='left' )

In [7]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = df_merged[df_merged['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,tac_torax_covid,condicion,region_separacion,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,Región de Aysén del General Carlos Ibáñez del ...,Coyhaique,Coyhaique,Coyhaique,11101.0,28763.0,29055.0,57818.0


In [8]:
data_aysen_filtered = data_aysen[(data_aysen['estado_caso']!='No validada')&(
                          data_aysen['vigente_no_eliminado']==True)&(
                          data_aysen['seremi']=='SEREMI De Aisén del General Carlos Ibañez del Campo')
                       ].rename(columns={'edad_hombres':'pop_hombres',
                                         'edad_mujeres':'pop_mujeres',
                                         'total':'pop_total'})

data_aysen_filtered.sample(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,tac_torax_covid,condicion,region_separacion,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,pop_hombres,pop_mujeres,pop_total
2737,1049208,999208,50,Coronavirus (COVID-19),26,2020-06-27,DESCARTADA,Centro de Salud Familiar Puerto Aysen,Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,Región de Aysén del General Carlos Ibáñez del ...,Aysen,Aysen,Aysen,11201.0,12719.0,11240.0,23959.0


In [9]:
# Transforma Fechas a formato DateTime 

data_aysen_filtered['fecha_notificacion'] = pd.to_datetime(data_aysen_filtered['fecha_notificacion'], errors = 'coerce') 

data_aysen_filtered['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_1'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_2'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_3'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_1'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_2'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_3'], errors = 'coerce')

In [10]:
# Muestra variables presente en DB en formato DF

s=pd.DataFrame(data_aysen_filtered.columns)

s[s[0]=='contactado']


,0


In [11]:
# Selecciona variables de importancia

data_aysen_clean =  data_aysen_filtered[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'tipo_caso_busqueda',
                                'establecimiento_salud',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3',
                                'estado_paciente',
                                'lugar_reposo',
                                'estado_caso',
                                'presentacion_clinica',
                                'contacto_caso_sospechoso',
                                'motivo_examen',
                                'hospitalizacion',
                                'trabajador_salud',
                                'caso_cerrado',
                                'fecha_primeros_sintomas',
                                'fecha_diagnostico',
                                'comuna_seremi_xi',
                                'provincia_seremi_xi',
                                'pop_hombres',
                                'pop_mujeres',
                                'pop_total'
                               ]]

In [12]:
# Agrega 53 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)


<ipython-input-12-d2f259f8832e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
<ipython-input-12-d2f259f8832e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53


In [14]:
data_aysen_clean.sample(10)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,tipo_caso_busqueda,establecimiento_salud,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,...,hospitalizacion,trabajador_salud,caso_cerrado,fecha_primeros_sintomas,fecha_diagnostico,comuna_seremi_xi,provincia_seremi_xi,pop_hombres,pop_mujeres,pop_total
13835,3066073,Coyhaique,2020-09-28,40,DESCARTADA,Atención médica,Hospital Regional (Coihaique),NaN,NaN,NaN,...,NaN,SI,False,NaN,2020-09-29,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
30122,4981963,Aisén,2020-11-25,48,DESCARTADA,Búsqueda activa,Ejercicio libre de la profesión XI región,NaN,NaN,NaN,...,NaN,NO,False,NaN,2020-11-26,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,NaN,NaN,NaN
64026,8485393,Coyhaique,2021-02-22,8,CONFIRMADA,Atención médica,Ejercicio libre de la profesión XI región,NaN,NaN,NaN,...,NaN,NO,False,2021-02-20,2021-02-22,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,NaN,NaN,NaN
39937,6096725,Coyhaique,2020-12-27,53,DESCARTADA,Atención médica,Hospital Regional (Coihaique),NaN,NaN,NaN,...,NaN,NO,False,NaN,2020-12-28,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
156121,16303736,Aisén,2021-08-23,34,DESCARTADA,Búsqueda activa,Ejercicio libre de la profesión XI región,NEGATIVO,NaN,NaN,...,NaN,NO,False,NaN,2021-08-24,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,NaN,NaN,NaN
11380,2817316,Coyhaique,2020-09-19,38,DESCARTADA,Atención médica,Consultorio Alejandro Gutiérrez,NEGATIVO,NaN,NaN,...,NaN,NO,False,NaN,NaN,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
92447,11373705,Aisén,2021-04-24,16,DESCARTADA,Búsqueda activa,Centro de Salud Familiar Puerto Aysen,NEGATIVO,NaN,NaN,...,NaN,NO,False,NaN,NaN,Aysen,Aysen,12719.0,11240.0,23959.0
4394,1406227,Coyhaique,2020-07-24,30,DESCARTADA,Atención médica,Centro de Salud Familiar Puerto Aysen,NEGATIVO,NaN,NaN,...,NaN,SI,False,2020-07-21,NaN,Aysen,Aysen,12719.0,11240.0,23959.0
34808,5505705,Coyhaique,2020-12-10,50,DESCARTADA,Búsqueda activa,Actividades gestionadas por la Dirección del S...,NEGATIVO,NaN,NaN,...,NaN,DESCONOCIDO,False,NaN,2020-12-13,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
148320,15738034,Aisén,2021-08-06,31,DESCARTADA,Búsqueda activa,Centro de Salud Familiar Puerto Aysen,NaN,NEGATIVO,NaN,...,NaN,NO,False,NaN,NaN,Aysen,Aysen,12719.0,11240.0,23959.0


# Indicador 5: Oportunidad en la investigación epidemiológica de casos

## Objetivo: Fortalecer la investigación epidemiológica de los casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala Territorial:** Comunal, Regional, Nacional

**Fuente de Información:** Base EPIVIGILIA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:comuna\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:región\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso notificado que cumple los requisitos para estar en el modulo de seguimiento
- La fecha para determinar el inicio de seguimiento corresponde a la fecha de resultado (caso confirmado) o fecha de notificación (caso probable). Para un caso confirmado, si el resultado del test es anterior a la fecha de notificación, se debe iniciar seguimiento a partir de la fecha en que el registro está disponible en la bandeja de seguimiento.
- Se cumple el indicador si, dentro de las 48 hrs. a partir de la fecha de inicio de seguimiento se tiene al menos una actividad de seguimiento exitosa (se logra ubicar al caso).
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento de notificación.
- Se consideran sólo registros de casos que estén activos o egresados en la bandeja de seguimiento.

# Indicador 6: Capacidad de investigación de contactos

## Objetivos: Fortalecer la investigación epidemiológica de los contactos estrechos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

**Cálculo del indicador a nivel comunal**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:comuna\:1er\:establecimiento\:semana\:x}$

**Cálculo del indicador a nivel regional**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:región\:1er\:establecimiento\:semana\:x}$

**Observaciones**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso nuevo notificado que cumple los requisitos para estar en el módulo de seguimiento
- Los casos nuevos con contactos estrechos son considerados de forma dicotómica (sí o no)
- El primer establecimiento corresponde al establecimiento de notificación. Se seguirá considerando el primer establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica.
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento

# Indicador 7: Capacidad de investigación de contactos

## Objetivo: Evaluar la capacidad de seguimiento de contactos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento.

**Cálculo del indicador a nivel comunal**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

**Cálculo del indicador a nivel regional**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observaciones**

- No se incluyen los contactos nuevos no notificados
- Se considera como registro de contacto aquel que cumple las condiciones para estar en la bandeja de seguimiento
- Se consideran sólo registros de casos activos o egresados en la bandeja de seguimiento
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica

# Indicador 8: Casos en seguimiento

## Objetivo: Evaluar la capacidad de seguimiento de casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de Información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:región\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados ("casos reportados por laboratorio")
- Se considera un caso activo como aquel caso notificado que cumple la definición de caso activo, con fecha de notificación en la semana de interés
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento
- Según protocolo implica un seguimiento diario por día de seguimineto, desde el inicio de seguimiento y se valora de forma dicotómica (cumple o no cumple el protocolo de seguimiento)
- Por seguimiento diario se entiende que diariamente, se debe realizar una actividad de seguimiento (la cual puede ser exitosa o no)

# Indicador 10: Contactos con al menos 2 seguimientos dentro de su periodo de cuarentena de 14 días

## Objetivo: Evaluar la capacidad de seguimiento de contactos estrechos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:región\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- El indicador se construye sobre los contactos que egresan (fecha de término de cuarentena) de seguimiento, durante la semana de interés (esto es, sólo registros egresados de la bandeja de seguimiento)
- El periodo de seguimiento para el contacto corresponde a la cantidad de días entre la 1era fecha en que el contacto está disponible en la bandeja de seguimiento hasta la última fecha fin cuarentena
- Se cumple con el indicador si en el período de seguimiento del contacto se registran al menos 2 actividades de seguimiento (las cuales pueden ser exitosas o no)
- Se excluyen los contactos que se transforman en caso y que están notificados (tiene folio asociado como caso)